In [52]:
company = 'BABA'
freq = 'daily'
num_epochs = 50 
path = company + "_" + freq + ".csv"

In [53]:
import pandas as pd
from yfinance import yfinance as yf
msft = yf.Ticker("FB")
# get historical market data
hist = msft.history(period="max")
df = (hist[['Open','High','Low','Close','Volume']])
df=df.sort_index(ascending=False)
df.index.names = ['date']
df.to_csv(path)
from pprint import pprint
pprint(df.head(10))

                  Open        High         Low       Close    Volume
date                                                                
2021-01-29  265.299988  266.559998  258.160004  258.964996   5136755
2021-01-28  277.179993  286.790009  264.700012  265.000000  37647200
2021-01-27  282.529999  283.450012  268.119995  272.140015  35346200
2021-01-26  278.140015  285.390015  277.809998  282.049988  19373600
2021-01-25  278.140015  280.100006  271.510010  278.010010  19087000
2021-01-22  272.010010  278.470001  272.000000  274.500000  21910000
2021-01-21  269.260010  273.600006  267.489990  272.869995  20838700
2021-01-20  268.929993  270.320007  263.600006  267.480011  25199900
2021-01-19  256.899994  262.200012  252.720001  261.100006  28028500
2021-01-15  247.899994  253.860001  247.160004  251.360001  24786100


In [ ]:
!python3 tech_ind_model.py $company $freq $num_epochs

2021-01-29 15:09:22.261268: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-01-29 15:09:22.261352: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
(1924, 50, 5)
(214, 50, 5)
2021-01-29 15:09:24.678668: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-01-29 15:09:24.679039: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-01-29 15:09:24.679100: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-01-29 15:09:24.679141: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver doe

In [ ]:
import numpy as np
from keras.models import load_model
from util import csv_to_dataset, history_points

model = load_model('technical_model' + company + "_" + freq + '.h5')

ohlcv_histories, technical_indicators, next_day_open_values, unscaled_y, y_normaliser = csv_to_dataset(path)

test_split = 0.9
n = int(ohlcv_histories.shape[0] * test_split)

ohlcv_train = ohlcv_histories[:n]
tech_ind_train = technical_indicators[:n]
y_train = next_day_open_values[:n]

ohlcv_test = ohlcv_histories[n:]
tech_ind_test = technical_indicators[n:]
y_test = next_day_open_values[n:]

unscaled_y_test = unscaled_y[n:]

y_test_predicted = model.predict([ohlcv_test, tech_ind_test])
y_test_predicted = y_normaliser.inverse_transform(y_test_predicted)

buys = []
sells = []
thresh = 0.1

start = 0
end = -1

x = -1

In [ ]:
import pandas as pd
from yfinance import yfinance as yf
msft = yf.Ticker("FB")
# get stock info
msft.info
# get historical market data
hist = msft.history(period="max")
hist['Close'].plot(y="Close", use_index=True, figsize=(20,10))




data = pd.read_csv(path)['date'].tolist()
dates = data[0:len(unscaled_y_test[start:end])]
dates.reverse()
dataframe = {'Date': dates,
        'Real': [x[0] for x in unscaled_y_test[start:end].tolist()],
        'Predicted': [x[0] for x in y_test_predicted[start:end].tolist()]}
df = pd.DataFrame(dataframe, columns=['Date', 'Real', 'Predicted'])
df.plot(x ='Date', y=['Real', 'Predicted'], kind = 'line', figsize=(20,10))

In [ ]:
import matplotlib.pyplot as plt

plt.gcf().set_size_inches(22, 15, forward=True)

real = plt.plot(unscaled_y_test[start:end], label='real')
pred = plt.plot(y_test_predicted[start:end], label='predicted')

plt.legend(['Real', 'Predicted'])

plt.show()

In [ ]:
df